In [12]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark_Hadoop")\
.master("local[*]")\
.config("spark.cores.max", 2) \
.config("spark.executor.cores", 1)\
.config("spark.sql.autoBroadcastJoinThreshold", "-1")\
.config("spark.sql.adaptive.enabled", False)\
.config("spark.sql.adaptive.coalescePartitions.enabled", False)\
.config("spark.sql.adaptive.skewJoin.enabled", False)\
.config("spark.sql.catalogImplementation", 2)\
.getOrCreate()

In [17]:
spark.conf.get("spark.sql.catalogImplementation")
spark.sql("show databases").show()
spark.sql("use default")
spark.sql("show tables").show()


+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [18]:
transaction_df = spark.read.format("csv").option("header", True).load("file:////home/dominic/Desktop/pythonLearning/csvFiles/practice_Data/Interview_Preparation/transactions.csv")

users_df = spark.read.format("csv").option("header", True).load("file:////home/dominic/Desktop/pythonLearning/csvFiles/practice_Data/Interview_Preparation/users.csv")

In [19]:
transaction_df.createOrReplaceTempView("transactions")
users_df.createOrReplaceTempView("users")
spark.sql("show tables in default").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |transactions|       true|
|         |       users|       true|
+---------+------------+-----------+



In [21]:
trans_table = spark.sql("select * from transactions")
uaer_table = spark.sql("select * from users")

In [28]:
user_df = spark.sql("""
    select * from transactions t
    join users u
    on t.user_id = u.user_id
""")
user_df.show()

+------+-------+------+----------+-------+-----+-----------------+
|txn_id|user_id|amount|      date|user_id| name|            email|
+------+-------+------+----------+-------+-----+-----------------+
|    T1|    101|   250|2025-06-01|    101|Alice|alice@example.com|
|    T2|    101|   100|2025-06-01|    101|Alice|alice@example.com|
|    T4|    101|   400|2025-06-03|    101|Alice|alice@example.com|
|    T6|    104|   150|2025-06-04|    104|David|david@example.com|
|    T3|    102|   300|2025-06-02|    102|  Bob|  bob@example.com|
|    T5|    103|   200|2025-06-04|    103|Carol|carol@example.com|
+------+-------+------+----------+-------+-----+-----------------+



In [10]:
joined_df = transaction_df.join(users_df, transaction_df.user_id == users_df.user_id, "left")

In [11]:
joined_df.write.format("noop").mode("overwrite").save()